# ИД23-1 Маслов АН
## Контрольная работа по ТОБД
### Вариант 18

### 1. Файл stock_data.csv содержит информацию о котировках акций за некоторый период. В колонках Date, Open, High, Low, Close, Volume задаются соответственно день, цена открытия, максимум цены, минимум цены, цена закрытия, объем продаж. Цена открытия может отличаться от цены закрытия предыдущего дня. Определить в каком диапазоне изменялясь разница между этими значениями (учитывать только пары соседних дней). Решить задачу средствами dask.DataFrame, используя возможности Dask по повышению производительности. Проверить работу программы на тестовом примере.

In [ ]:
import dask.dataframe as dd

df = dd.read_csv(
    "stock_data.csv",
    sep=";",
    assume_missing=True,
    blocksize="64MB"   # параллельное чтение
)

df["Prev_Close"] = df["Close"].shift(1)
df["Diff"] = df["Open"] - df["Prev_Close"]

diff = df["Diff"].dropna()

min_diff, max_diff = dd.compute(diff.min(), diff.max())

print(f"Диапазон: [{min_diff}, {max_diff}]")



Диапазон разниц между Open и предыдущим Close: [-42.16, 9.5]


: 

### 2. Создать базу данных SQLite и таблицу в ней со структурой, соответствующей файлу stock_data.csv. Загрузить данные их файла в БД. Решить задачу из 1-го вопроса, использую SQL-запрос.

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("stocks.db")
df = pd.read_csv("stock_data.csv", sep=";")
df.to_sql("stock_data", conn, if_exists="replace", index=False)
conn.close()


In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("stocks.db")
query = """
SELECT
    MIN(Open - prev_close) AS min_diff,
    MAX(Open - prev_close) AS max_diff
FROM (
    SELECT
        Date,
        Open,
        LAG(Close) OVER (ORDER BY Date) AS prev_close
    FROM stock_data
)
WHERE prev_close IS NOT NULL;
"""
result = pd.read_sql_query(query, conn)
print( "Диапазон разниц между Open и предыдущим Close: \n", result)
conn.close()


Диапазон разниц между Open и предыдущим Close: 
    min_diff  max_diff
0    -42.16       9.5
